In [1]:
import xml.etree.ElementTree as ET
import glob

In [2]:
import time
import math
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
# we will work with totImages images
totImages = 150

In [5]:
fn = glob.glob("VOC2012/Annotations/*");

In [6]:
images = dict()
cat = set()

In [7]:
start = time.time()

cnt = 0

for f in fn:
    cnt = cnt + 1
    if cnt > totImages:
        break
    
    tree = ET.parse(f)
    root = tree.getroot()

    curFile = ""
    for child in root:
        if child.tag == 'filename':
            curFile = child.text
    images[curFile] = []
    
    for child in root:
        if child.tag == 'object':
            for child2 in child:
                if child2.tag == 'name':
                    images[curFile].append(child2.text)
                    cat.add(child2.text)
                    
print 'Done parsing in ', time.time() - start, " seconds"

Done parsing in  0.123862028122  seconds


In [8]:
cat = list(cat)
print cat

['sheep', 'horse', 'bicycle', 'bottle', 'cow', 'sofa', 'dog', 'bus', 'cat', 'person', 'train', 'diningtable', 'aeroplane', 'car', 'pottedplant', 'tvmonitor', 'chair', 'bird', 'boat', 'motorbike']


In [9]:
len(cat)

20

In [10]:
catimg = dict()
for curCat in cat:
    catimg[curCat] = []

for img in images:
    for curCat in set(images[img]):
        catimg[curCat].append(img)
        
print catimg['sheep'][:5]

['2007_001872.jpg', '2007_000925.jpg', '2007_000676.jpg', '2007_001733.jpg', '2007_000175.jpg']


In [11]:
featvec = images.copy()

cnt = 0

for img in images:
    cnt = cnt + 1
    if cnt <= 5:
        print img, images[img]
    
    features = featvec[img]
    
    featvec[img] = [0 for i in range(len(cat))]
    for f in features:
        for j in range(len(cat)):
            if cat[j] == f:
                featvec[img][j] = featvec[img][j] + 1
                break
                
    featvec[img] = np.array(featvec[img])
                
    if cnt <= 5:
        print img, featvec[img]
        print ""

2007_001154.jpg ['sofa', 'pottedplant']
2007_001154.jpg [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

2007_000645.jpg ['bird', 'bird']
2007_000645.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0]

2007_000061.jpg ['boat', 'boat']
2007_000061.jpg [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]

2007_000847.jpg ['diningtable', 'person']
2007_000847.jpg [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0]

2007_000129.jpg ['bicycle', 'bicycle', 'bicycle', 'person', 'person', 'person']
2007_000129.jpg [0 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0]



In [12]:
# we will pick the exemplar such that max(nrm(v,v') < nrm(v, exemplar)). 
# taking the exemplar twice(or 3x, 4x) the longest vector should thus suffice
exemplar = [0]
for elem in featvec:
    if np.linalg.norm(featvec[elem]) > np.linalg.norm(exemplar):
        exemplar = featvec[elem][:]
exemplar = exemplar * 0 # the above constraint should be satisfied?

# compute a dictionary of norms and normdistances
exempDist = featvec.copy()
for elem in featvec:
    exempDist[elem] = np.linalg.norm(featvec[elem] - exemplar)

dist = dict()
for el1 in featvec:
    for el2 in featvec:
        dist[(el1, el2)] = np.linalg.norm(featvec[el1] - featvec[el2])

In [13]:
from replacementGreedy import replacementGreedy
from localsearch import localsearch
from greedysum import greedysum
from greedymerge import greedymerge
from greedyclustering import greedyclustering

In [14]:
n = len(images.keys())
m = len(cat)
l = 20

In [15]:
solution = []
runtime = []
calls = []

Kvalues = [1,2,4,5,7]
for k in range(10,l + 5,5):
    Kvalues.append(k)

for k in Kvalues:
    rgTime = time.time()
    rgS, rgCost, rgEvals = replacementGreedy(l, k, featvec, exempDist, dist, catimg)
    rgTime = time.time() - rgTime
    
    lsTime = time.time()
    lsS, lsCost, lsEvals = localsearch(l, k, featvec, exempDist, dist, catimg, 0.2)
    lsTime = time.time() - lsTime
    
    gsTime = time.time()
    gsS, gsCost, gsEvals = greedysum(l, k, featvec, exempDist, dist, catimg)
    gsTime = time.time() - gsTime
    
    gmTime = time.time()
    gmS, gmCost, gmEvals = greedymerge(l, k, featvec, exempDist, dist, catimg)
    gmTime = time.time() - gmTime
    
#     clS, clCost, clEvals = greedyclustering(l, k, featvec, exempDist, dist, catimg)
    
    solution.append([rgCost, lsCost, gsCost, gmCost])
    runtime.append([rgTime, lsTime, gsTime, gmTime])
    calls.append([rgEvals, lsEvals, gsEvals, gmEvals])
    
    print ""
    print "Done for k = ", k
    print "\n\n\n"

Local search value after initialization =  20.5025280019
Intermediate cost at step  0  =  20.5025280019
Local Search gives cost =  20.5025280019
Greedy Sum gives cost =  18.9566578104
Greedy Merge gives cost =  20.5025280019
Size of S is  20

Done for k =  1




Local search value after initialization =  26.5882651959
Intermediate cost at step  0  =  26.5882651959
Local Search gives cost =  26.5882651959
Greedy Sum gives cost =  24.8642997554
Greedy Merge gives cost =  29.9774833006
Size of S is  37

Done for k =  2




Local search value after initialization =  28.3674511714
Intermediate cost at step  0  =  28.3674511714
Local Search gives cost =  28.3674511714
Greedy Sum gives cost =  28.0367150114
Greedy Merge gives cost =  39.9292418778
Size of S is  67

Done for k =  4




Local search value after initialization =  28.488694076
Intermediate cost at step  0  =  28.488694076
Local Search gives cost =  28.488694076
Greedy Sum gives cost =  28.4776680307
Greedy Merge gives cost =  43.

In [16]:
# store locally
filename = open('../data/voc2012-L.txt', 'w')

print>>filename, 'solution'
for item in solution:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'runtime'
for item in runtime:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'calls'
for item in calls:
    print>>filename,item[0],item[1],item[2],item[3]

In [20]:
plt.clf()

ax = plt.subplot(111)

fs = 17
lfs=22

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('k', fontsize=lfs)
ax.set_ylabel('Objective Value', fontsize=lfs)

plt.ylim(15,50)
plt.xlim(min(Kvalues), max(Kvalues))

# ax.set_xticks(range(min(Kvalues),max(Kvalues)),1)
xticks = [min(Kvalues)]
for k in range(5, max(Kvalues) + 5, 5):
    xticks.append(k)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['ro', 'cs', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Local Search', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(solution)):
        vals.append(solution[i][ind])
        
    if ind != 0:
        plt.plot(Kvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])
    else:
        plt.plot(Kvalues, vals, 'r--', linewidth=4, label = labelNames[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize(fs)

plt.savefig("../../writeup/images/voc2012-fixed-l")

plt.close()

In [21]:
plt.clf()

ax = plt.subplot(111)

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('k', fontsize=lfs)
ax.set_ylabel('Log(runtime)', fontsize=lfs)

plt.ylim(-10,25)
plt.xlim(min(Kvalues), max(Kvalues))

xticks = [min(Kvalues)]
for l in range(10, max(Kvalues) + 10, 10):
    xticks.append(l)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['ro', 'cs', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Local Search', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(runtime)):
        vals.append(math.log(runtime[i][ind], 2))
        
    if ind != 0:
        plt.plot(Kvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])
    else:
        plt.plot(Kvalues, vals, 'r--', linewidth=4, label = labelNames[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='upper right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize(fs)
    
plt.savefig("../../writeup/images/voc2012-runtime-fixed-l")

plt.close()